In [ ]:
!pip install transformers
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 15.9 MB/s eta 0:00:00


In [ ]:
import os
#import yaml
import pandas as pd
import numpy as np
import torch
import warnings
import seaborn as sns
import transformers
from transformers import AutoTokenizer, AutoModel,get_cosine_schedule_with_warmup

device = torch.device("cuda")


In [ ]:
from transformers import AutoConfig

In [ ]:
import pytorch_lightning as pl

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
complete_df = pd.read_csv('bert_complete_cleaned.csv')

In [ ]:
complete_df.head()

,tweet,cleaned,final_cleaned,labels,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,@cath__kath AstraZeneca is made with the kidne...,username astrazeneca is made with the kidney c...,Vaccine is made with the kidney cells of a lit...,ingredients,0,0,0,1,0,0,0,0,0,0,0,0
1,It begins. Please find safe alternatives to th...,it begins. please find safe alternatives to th...,It begins. Please find safe alternatives to th...,side-effect,0,0,0,0,0,0,0,0,0,0,1,0
2,"@PaolaQP1231 Well, I mean congratulations Covi...","username well, i mean congratulations covid19 ...","Well, I mean congratulations disease for being...",side-effect,0,0,0,0,0,0,0,0,0,0,1,0
3,@BorisJohnson for those of us that do not wish...,username for those of us that do not wish a va...,for those of us that do not wish a vaccine so ...,mandatory,0,0,0,0,1,0,0,0,0,0,0,0
4,She has been trying to speak out: writing lett...,she has been trying to speak out: writing lett...,She has been trying to speak out writing lette...,side-effect rushed,0,0,0,0,0,0,0,0,0,1,1,0


In [ ]:
PRE_TRAINED_MODEL_NAME = 'digitalepidemiologylab/covid-twitter-bert-v2'

## CHOOSING TOKEN LENGTH

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

## PARAMETERS

In [ ]:
MAX_LEN = 120
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 2e-5

## DATASET

In [ ]:
target_list = ['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory','none', 'pharma', 'political', 'religious', 'rushed', 'side-effect','unnecessary']

In [ ]:
label_count = {}
for target in target_list:
    label_count[target] = complete_df[target].sum()

In [ ]:
del label_count

In [ ]:
import gc
gc.collect()

71

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['cleaned']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
class TestDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['cleaned']
        #self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
        }

In [ ]:
train_size = 0.8
train_df = complete_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = complete_df.drop(train_df.index).reset_index(drop=True)

In [ ]:
#class frequency for db-loss
from collections import Counter
term2count = Counter([x for labels in train_df['labels'] for x in labels.split()])
class_freq = [term2count[x] for x in target_list]

In [ ]:
# gamma calculation for db-loss
r_all = []
for docu in train_df['labels']:
    docu_p = [1/term2count[x] for x in docu.split()]
    docu_p_sum = sum(docu_p)
    r_all.extend([p/docu_p_sum for p in docu_p])
gamma = (np.mean(r_all) + np.median(r_all)) / 2
gamma

0.907025641025641

In [ ]:
train_num = len(train_df)

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

## DATALOADER

### Loader

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=0
)

### CONFIG

In [ ]:
config = {
    'model_name': 'digitalepidemiologylab/covid-twitter-bert-v2',
    'n_labels': len(target_list),
    'batch_size': 16,
    'lr': 2e-5,
    'warmup': 0.2,
    'train_size': len(train_data_loader),
    'weight_decay': 0.001,
    'n_epochs': 15,
    'max_len' : MAX_LEN,
    'dropout' : 0.2
}

## DATA MODULE

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
class Custom_Data_Module(pl.LightningDataModule):

    def __init__(self, train_path, val_path, pred_path = None, attributes = None, batch_size: int = 16, max_token_length: int = 128,  model_name='digitalepidemiologylab/covid-twitter-bert-v2'):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.pred_path = pred_path
        self.attributes = attributes
        self.batch_size = batch_size
        self.max_token_length = max_token_length
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def setup(self, stage = None):
        if stage in (None, "fit"):
            self.train_dataset = CustomDataset(self.train_path, max_len=config['max_len'], tokenizer=self.tokenizer)
            self.val_dataset = CustomDataset(self.val_path, max_len=config['max_len'], tokenizer=self.tokenizer)
        if stage == 'predict':
            self.pred_dataset = TestDataset(self.pred_path, max_len=config['max_len'], tokenizer=self.tokenizer)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers=4, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=4, shuffle=False)

    def predict_dataloader(self):
        return DataLoader(self.pred_dataset, batch_size = self.batch_size, num_workers=4, shuffle=False)


## MODEL ARCHITECTURE

In [ ]:
import torch.nn as nn
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

In [ ]:
from torch.optim import AdamW

### DB LOSS

In [ ]:
class ResampleLoss(nn.Module):

    def __init__(self,
                 use_sigmoid=True, partial=False,
                 loss_weight=1.0, reduction='mean',
                 reweight_func=None,  # None, 'inv', 'sqrt_inv', 'rebalance', 'CB'
                 weight_norm=None, # None, 'by_instance', 'by_batch'
                 focal=dict(
                     focal=True,
                     alpha=0.5,
                     gamma=2,
                 ),
                 map_param=dict(
                     alpha=10.0,
                     beta=0.2,
                     gamma=0.1
                 ),
                 CB_loss=dict(
                     CB_beta=0.9,
                     CB_mode='average_w'  # 'by_class', 'average_n', 'average_w', 'min_n'
                 ),
                 logit_reg=dict(
                     neg_scale=5.0,
                     init_bias=0.1
                 ),
                 class_freq=None,
                 train_num=None):
        super(ResampleLoss, self).__init__()

        assert (use_sigmoid is True) or (partial is False)
        self.use_sigmoid = use_sigmoid
        self.partial = partial
        self.loss_weight = loss_weight
        self.reduction = reduction
        if self.use_sigmoid:
            if self.partial:
                self.cls_criterion = partial_cross_entropy
            else:
                self.cls_criterion = binary_cross_entropy
        else:
            self.cls_criterion = cross_entropy

        # reweighting function
        self.reweight_func = reweight_func

        # normalization (optional)
        self.weight_norm = weight_norm

        # focal loss params
        self.focal = focal['focal']
        self.gamma = focal['gamma']
        self.alpha = focal['alpha'] # change to alpha

        # mapping function params
        self.map_alpha = map_param['alpha']
        self.map_beta = map_param['beta']
        self.map_gamma = map_param['gamma']

        # CB loss params (optional)
        self.CB_beta = CB_loss['CB_beta']
        self.CB_mode = CB_loss['CB_mode']

        self.class_freq = torch.from_numpy(np.asarray(class_freq)).float().cuda()
        self.num_classes = self.class_freq.shape[0]
        self.train_num = train_num # only used to be divided by class_freq
        # regularization params
        self.logit_reg = logit_reg
        self.neg_scale = logit_reg[
            'neg_scale'] if 'neg_scale' in logit_reg else 1.0
        init_bias = logit_reg['init_bias'] if 'init_bias' in logit_reg else 0.0
        self.init_bias = - torch.log(
            self.train_num / self.class_freq - 1) * init_bias ########################## bug fixed https://github.com/wutong16/DistributionBalancedLoss/issues/8

        self.freq_inv = torch.ones(self.class_freq.shape).cuda() / self.class_freq
        self.propotion_inv = self.train_num / self.class_freq

    def forward(self,
                cls_score,
                label,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):

        assert reduction_override in (None, 'none', 'mean', 'sum')
        reduction = (
            reduction_override if reduction_override else self.reduction)

        weight = self.reweight_functions(label)

        cls_score, weight = self.logit_reg_functions(label.float(), cls_score, weight)

        if self.focal:
            logpt = self.cls_criterion(
                cls_score.clone(), label, weight=None, reduction='none',
                avg_factor=avg_factor)
            # pt is sigmoid(logit) for pos or sigmoid(-logit) for neg
            pt = torch.exp(-logpt)
            wtloss = self.cls_criterion(
                cls_score, label.float(), weight=weight, reduction='none')
            alpha_t = torch.where(label==1, self.alpha, 1-self.alpha)
            loss = alpha_t * ((1 - pt) ** self.gamma) * wtloss ####################### balance_param should be a tensor
            loss = reduce_loss(loss, reduction)             ############################ add reduction
        else:
            loss = self.cls_criterion(cls_score, label.float(), weight,
                                      reduction=reduction)

        loss = self.loss_weight * loss
        return loss

    def reweight_functions(self, label):
        if self.reweight_func is None:
            return None
        elif self.reweight_func in ['inv', 'sqrt_inv']:
            weight = self.RW_weight(label.float())
        elif self.reweight_func in 'rebalance':
            weight = self.rebalance_weight(label.float())
        elif self.reweight_func in 'CB':
            weight = self.CB_weight(label.float())
        else:
            return None

        if self.weight_norm is not None:
            if 'by_instance' in self.weight_norm:
                max_by_instance, _ = torch.max(weight, dim=-1, keepdim=True)
                weight = weight / max_by_instance
            elif 'by_batch' in self.weight_norm:
                weight = weight / torch.max(weight)

        return weight

    def logit_reg_functions(self, labels, logits, weight=None):
        if not self.logit_reg:
            return logits, weight
        if 'init_bias' in self.logit_reg:
            logits += self.init_bias
        if 'neg_scale' in self.logit_reg:
            logits = logits * (1 - labels) * self.neg_scale  + logits * labels
            if weight is not None:
                weight = weight / self.neg_scale * (1 - labels) + weight * labels
        return logits, weight

    def rebalance_weight(self, gt_labels):
        repeat_rate = torch.sum( gt_labels.float() * self.freq_inv, dim=1, keepdim=True)
        pos_weight = self.freq_inv.clone().detach().unsqueeze(0) / repeat_rate
        # pos and neg are equally treated
        weight = torch.sigmoid(self.map_beta * (pos_weight - self.map_gamma)) + self.map_alpha
        return weight

    def CB_weight(self, gt_labels):
        if  'by_class' in self.CB_mode:
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
        elif 'average_n' in self.CB_mode:
            avg_n = torch.sum(gt_labels * self.class_freq, dim=1, keepdim=True) / \
                    torch.sum(gt_labels, dim=1, keepdim=True)
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, avg_n)).cuda()
        elif 'average_w' in self.CB_mode:
            weight_ = torch.tensor((1 - self.CB_beta)).cuda() / \
                      (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
            weight = torch.sum(gt_labels * weight_, dim=1, keepdim=True) / \
                     torch.sum(gt_labels, dim=1, keepdim=True)
        elif 'min_n' in self.CB_mode:
            min_n, _ = torch.min(gt_labels * self.class_freq +
                                 (1 - gt_labels) * 100000, dim=1, keepdim=True)
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, min_n)).cuda()
        else:
            raise NameError
        return weight

    def RW_weight(self, gt_labels, by_class=True):
        if 'sqrt' in self.reweight_func:
            weight = torch.sqrt(self.propotion_inv)
        else:
            weight = self.propotion_inv
        if not by_class:
            sum_ = torch.sum(weight * gt_labels, dim=1, keepdim=True)
            weight = sum_ / torch.sum(gt_labels, dim=1, keepdim=True)
        return weight


def reduce_loss(loss, reduction):
    """Reduce loss as specified.
    Args:
        loss (Tensor): Elementwise loss tensor.
        reduction (str): Options are "none", "mean" and "sum".
    Return:
        Tensor: Reduced loss tensor.
    """
    reduction_enum = F._Reduction.get_enum(reduction)
    # none: 0, elementwise_mean:1, sum: 2
    if reduction_enum == 0:
        return loss
    elif reduction_enum == 1:
        return loss.mean()
    elif reduction_enum == 2:
        return loss.sum()


def weight_reduce_loss(loss, weight=None, reduction='mean', avg_factor=None):
    """Apply element-wise weight and reduce loss.
    Args:
        loss (Tensor): Element-wise loss.
        weight (Tensor): Element-wise weights.
        reduction (str): Same as built-in losses of PyTorch.
        avg_factor (float): Avarage factor when computing the mean of losses.
    Returns:
        Tensor: Processed loss values.
    """
    # if weight is specified, apply element-wise weight
    if weight is not None:
        loss = loss * weight

    # if avg_factor is not specified, just reduce the loss
    if avg_factor is None:
        loss = reduce_loss(loss, reduction)
    else:
        # if reduction is mean, then average the loss by avg_factor
        if reduction == 'mean':
            loss = loss.sum() / avg_factor
        # if reduction is 'none', then do nothing, otherwise raise an error
        elif reduction != 'none':
            raise ValueError('avg_factor can not be used with reduction="sum"')
    return loss


def binary_cross_entropy(pred,
                         label,
                         weight=None,
                         reduction='mean',
                         avg_factor=None):

    # weighted element-wise losses
    if weight is not None:
        weight = weight.float()

    loss = F.binary_cross_entropy_with_logits(
        pred, label.float(), weight, reduction='none')
    loss = weight_reduce_loss(loss, reduction=reduction, avg_factor=avg_factor)

    return loss

### MODEL ARCHITECTURE

In [ ]:
bert_config = AutoConfig.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
class Tweet_Classifier(pl.LightningModule):

    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config['model_name'], config = bert_config)
        self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classifier = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.config['n_labels'])
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_func = ResampleLoss(reweight_func='rebalance', loss_weight=1.0,
                             focal=dict(focal=True, alpha=0.5, gamma=2),
                             logit_reg=dict(init_bias=0.05, neg_scale=2.0),
                             map_param=dict(alpha=0.1, beta=10.0, gamma=0.9),
                             class_freq=class_freq, train_num=train_num)
        self.dropout = nn.Dropout(config['dropout'])

    def forward(self, input_ids, attention_mask,token_type_ids,targets=None):
        # roberta layer
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask,return_dict = True)
        #pooled_output = torch.mean(output.last_hidden_state, 1)
        # final logits
        pooled_output = self.dropout(output.pooler_output)
        #pooled_output = self.hidden(pooled_output)
        #pooled_output = F.relu(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        # calculate loss
        loss = 0
        if targets is not None:
            loss = self.loss_func(logits.view(-1, self.config['n_labels']), targets.view(-1, self.config['n_labels']))
        return loss, logits

    def training_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("train loss ", loss, prog_bar = True, logger=True)
        return {"loss":loss, "predictions":outputs, "targets": batch["targets"]}

    def validation_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("validation loss", loss, prog_bar = True, logger=True)
        return {"val_loss": loss, "predictions":outputs, "targets": batch["targets"]}

    def predict_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        return outputs

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'], weight_decay=self.config['weight_decay'])
        total_steps = self.config['train_size']/self.config['batch_size']
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
          scheduler=scheduler,
          interval='step'
        )
      )

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
Custom_data_module = Custom_Data_Module(train_path = train_df, val_path = val_df, attributes=target_list, batch_size=config['batch_size'])
Custom_data_module.setup()

# model
model = Tweet_Classifier(config)

# trainer and fit
#trainer = pl.Trainer(max_epochs=config['n_epochs'], gpus=1, num_sanity_val_steps=50)
early_stopping = EarlyStopping(monitor="validation loss", patience=3, mode="min")
trainer = pl.Trainer(max_epochs=config['n_epochs'], accelerator='gpu', callbacks=[early_stopping],num_sanity_val_steps=50)
trainer.fit(model, Custom_data_module)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type         | Params
--------------------------------------------------
0 | pretrained_model | BertModel    | 335 M 
1 | hidden           | Linear       | 1.0 M 
2 | classifier       | Linear       | 12.3 K
3 | loss_func        | ResampleLoss | 0     
4 | dropout          | Dropout      | 0     
--------------------------------------------------
336 M     Trainable params
0         Non-trainable params
336 M     Total params
1,344.815 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint("ctbert2.ckpt")

## Testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm

In [ ]:
test_df = pd.read_csv('test_df_2.csv')
test_df

,id,tweet,cleaned,cleaned_tweet,final_cleaned
0,1070378532260470789t,Study Links HPV Vaccine to Historically High I...,study links hpv vaccine to historically high i...,Study Links HPV Vaccine to Historically High I...,Study Links disease Vaccine to Historically Hi...
1,973746711964372993t,Deaths from tainted measles vaccine affecting ...,deaths from tainted measles vaccine affecting ...,Deaths from tainted measles vaccine affecting ...,disease from tainted vaccine affecting anti-po...
2,1043031076787040257t,"Am apreciat un videoclip pe @YouTube, https://...","am apreciat un videoclip pe username , url tre...","Am apreciat un videoclip pe , Treatment and C...","Am apreciat un videoclip pe , Treatment and Cu..."
3,1066338147527741440t,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...,video --&gt;&gt; mmr #vaccine increase risk of...,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...,VIDEO --&gt;&gt ; MMR # Vaccine increase risk ...
4,963522018544152576t,Oral Polio Vaccine: Infecting Unvaccinated Kid...,oral polio vaccine: infecting unvaccinated kid...,Oral Polio Vaccine Infecting Unvaccinated Kid...,Oral vaccine Infecting Unvaccinated Kids ! via...
...,...,...,...,...,...
481,1099937642169405440t,I know it's hard to believe... But another vac...,i know it's hard to believe... but another vac...,I know it is hard to believe... But another va...,I know it is hard to believe ... But another v...
482,1536674508731518985t,Seizures on the Very Day of the MMR Vaccine: T...,seizures on the very day of the mmr vaccine: t...,Seizures on the Very Day of the MMR Vaccine T...,disease on date date date of date vaccine Tale...
483,1183314494874968064t,Kenyan Doctors Say UNICEF Is Making Women Barr...,kenyan doctors say unicef is making women barr...,Kenyan Doctors Say UNICEF Is Making Women Barr...,Kenyan Doctors Say organisation Is Making Wome...
484,1327581896243556352t,Not doing it. My experience with vaccines? In ...,not doing it. my experience with vaccines? in ...,Not doing it. My experience with vaccines? In ...,Not doing it . My experience with vaccines ? I...


In [ ]:
def classify_raw_comments(model, dm):
  trainer = pl.Trainer()
  predictions = trainer.predict(model, datamodule=dm)
  flattened_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in predictions for p in batch])
  return flattened_predictions

In [ ]:
Custom_data_module = Custom_Data_Module(train_path = train_df, val_path = val_df,pred_path = val_df, attributes=target_list, batch_size=config['batch_size'])

In [ ]:
predictions = classify_raw_comments(model, Custom_data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

In [ ]:
labels = val_df.iloc[:,4:]

In [ ]:
labels.columns

Index(['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory',
       'none', 'pharma', 'political', 'religious', 'rushed', 'side-effect',
       'unnecessary'],
      dtype='object')

In [ ]:
predictions

array([[0.06307387, 0.03547763, 0.04840631, ..., 0.95646495, 0.16940565,
        0.07429877],
       [0.01974258, 0.15340313, 0.11490054, ..., 0.10832575, 0.98370683,
        0.01426042],
       [0.06724264, 0.02224953, 0.97181195, ..., 0.13901976, 0.02684224,
        0.05804368],
       ...,
       [0.00696881, 0.14053075, 0.15208684, ..., 0.06047039, 0.05940208,
        0.02380704],
       [0.04247977, 0.04855661, 0.04899526, ..., 0.05604061, 0.97443116,
        0.10785826],
       [0.02633087, 0.02956093, 0.06488913, ..., 0.04266374, 0.19442932,
        0.04462785]], dtype=float32)

In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score,classification_report

In [ ]:
from tqdm import tqdm

In [ ]:
def optimise_f1_score(true_labels: np.ndarray, pred_labels: np.ndarray):
    best_med_th = 0.5
    true_bools = [tl == 1 for tl in true_labels]
    micro_thresholds = (np.array(range(1, 100)) / 100)
    f1_results, pre_results, recall_results = [], [], []
    for th in tqdm(micro_thresholds):
        pred_bools = [pl > th for pl in pred_labels]
        test_f1 = f1_score(true_bools, pred_bools, average="macro", zero_division=0)
        test_precision = precision_score(
            true_bools, pred_bools, average="macro", zero_division=0
        )
        test_recall = recall_score(
            true_bools, pred_bools, average="macro", zero_division=0
        )
        f1_results.append(test_f1)
        pre_results.append(test_precision)
        recall_results.append(test_recall)
    best_f1_idx = np.argmax(f1_results)
    return micro_thresholds[best_f1_idx]

In [ ]:
optimise_f1_score(labels, predictions)

100%|██████████| 99/99 [00:04<00:00, 24.07it/s]


0.51

In [ ]:
upper, lower = 1, 0

predictions = np.where(predictions> 0.51, upper, lower)

print(classification_report(
  labels,
  predictions,
  target_names=target_list,
  zero_division=0
))

              precision    recall  f1-score   support

  conspiracy       0.94      0.61      0.74       102
     country       0.93      0.86      0.89        43
 ineffective       0.89      0.88      0.88       323
 ingredients       0.89      0.85      0.87        80
   mandatory       0.92      0.91      0.91       159
        none       0.91      0.93      0.92       135
      pharma       0.83      0.86      0.85       244
   political       0.94      0.73      0.82       137
   religious       0.90      0.75      0.82        12
      rushed       0.88      0.91      0.90       297
 side-effect       0.93      0.89      0.91       746
 unnecessary       0.93      0.79      0.85       151

   micro avg       0.91      0.86      0.88      2429
   macro avg       0.91      0.83      0.86      2429
weighted avg       0.91      0.86      0.88      2429
 samples avg       0.92      0.90      0.90      2429



In [ ]:
Custom_data_module = Custom_Data_Module(train_path = train_df, val_path = val_df,pred_path = test_df, attributes=target_list, batch_size=config['batch_size'])

In [ ]:
Valid_data_module = Custom_Data_Module(train_path = train_df, val_path = val_df,pred_path = val_df, attributes=target_list, batch_size=config['batch_size'])

In [ ]:
mymodel = Tweet_Classifier(config)
new_model = mymodel.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/dataset/ctbert2.ckpt",config = config)

In [ ]:
test_predictions = classify_raw_comments(new_model, Custom_data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
val_predictions = classify_raw_comments(new_model, Valid_data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

In [ ]:
test_predictions

array([[0.01368641, 0.06328242, 0.04117724, ..., 0.0551233 , 0.99290764,
        0.01410516],
       [0.04603674, 0.08728195, 0.12920433, ..., 0.06666409, 0.890531  ,
        0.03855754],
       [0.01847217, 0.08216222, 0.02830523, ..., 0.02583515, 0.9937202 ,
        0.02396765],
       ...,
       [0.03661989, 0.06776479, 0.09195744, ..., 0.18426615, 0.9431858 ,
        0.04380484],
       [0.05952088, 0.06518453, 0.07617312, ..., 0.05120275, 0.84156626,
        0.16191661],
       [0.02307214, 0.04378784, 0.02855588, ..., 0.08961237, 0.9869359 ,
        0.05747938]], dtype=float32)

In [ ]:
test_predictions_2 = test_predictions.copy()

In [ ]:
val_predictions_df = pd.DataFrame(val_predictions, columns = ['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory','none', 'pharma', 'political', 'religious', 'rushed', 'side-effect','unnecessary'])

In [ ]:
val_predictions_df.head()

,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,0.063074,0.035478,0.048406,0.156026,0.418899,0.028758,0.081120,0.074318,0.183399,0.956465,0.169406,0.074299
1,0.019743,0.153403,0.114901,0.034722,0.035586,0.011669,0.012648,0.052654,0.011957,0.108326,0.983707,0.014260
2,0.067243,0.022250,0.971812,0.060522,0.049541,0.039741,0.042928,0.005991,0.019299,0.139020,0.026842,0.058044
3,0.055512,0.013209,0.037032,0.051633,0.013361,0.013284,0.177586,0.016006,0.023062,0.990408,0.163701,0.033364
4,0.030691,0.065896,0.135815,0.122789,0.051493,0.041023,0.059047,0.022499,0.035055,0.987331,0.246647,0.067730


In [ ]:
labels.head()

,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
f1s = {}
for i in labels.columns:
  f1s[i] = optimise_f1_score(labels[i].to_numpy(), val_predictions_df[i].to_numpy())

100%|██████████| 99/99 [00:01<00:00, 59.88it/s]


In [ ]:
target_list

['conspiracy',
 'country',
 'ineffective',
 'ingredients',
 'mandatory',
 'none',
 'pharma',
 'political',
 'religious',
 'rushed',
 'side-effect',
 'unnecessary']

In [ ]:
f1s

{'conspiracy': 0.28,
 'country': 0.55,
 'ineffective': 0.59,
 'ingredients': 0.49,
 'mandatory': 0.59,
 'none': 0.48,
 'pharma': 0.44,
 'political': 0.39,
 'religious': 0.43,
 'rushed': 0.62,
 'side-effect': 0.48,
 'unnecessary': 0.34}

In [ ]:
test_predictions_3 = test_predictions

In [ ]:
del test_predictions_2

In [ ]:
import gc
gc.collect()

660

In [ ]:
for i in range(len(test_predictions_3)):
  for j in range(len(test_predictions_3[i])):
    if test_predictions_3[i][j] > f1s[target_list[j]]:
      test_predictions_3[i][j] = 1


In [ ]:
for i in range(len(test_predictions_2)):
    for j in range(len(test_predictions_2[i])):
        if test_predictions[i][j]>0.51:
            test_predictions[i][j] = 1

array([0.01368641, 0.06328242, 0.04117724, ..., 0.08961237, 0.9869359 ,
       0.05747938], dtype=float32)

In [ ]:
str_pred_list = []
for pred in test_predictions_3:
    str_pred = ""
    for i in range(len(pred)):
        if pred[i] == 1:
            str_pred = str_pred + target_list[i] + " "
    if str_pred == "":
        str_pred = target_list[np.argmax(pred)]
    str_pred_list.append(str_pred.strip())

In [ ]:
len(str_pred_list)

486

In [ ]:
test_df_2 = test_df[['id']]

In [ ]:
test_df_2['preds'] = str_pred_list

<ipython-input-100-daf958b10042>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_2['preds'] = str_pred_list


In [ ]:
test_df_2.to_csv('trial_check_2.csv',index = False)

In [ ]:
test_df

In [ ]:
val_df.columns

Index(['tweet', 'cleaned', 'final_cleaned', 'labels', 'conspiracy', 'country',
       'ineffective', 'ingredients', 'mandatory', 'none', 'pharma',
       'political', 'religious', 'rushed', 'side-effect', 'unnecessary'],
      dtype='object')

In [ ]:
label_class = ['conspiracy', 'country','ineffective', 'ingredients', 'mandatory', 'none', 'pharma','political', 'religious', 'rushed', 'side-effect', 'unnecessary']

In [ ]:
f1s = {}
for i in label_class:
  labels = val_df[i].to_numpy()


In [ ]:
test_dataset = TestDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
print(new_model.config)

{'model_name': 'digitalepidemiologylab/covid-twitter-bert-v2', 'n_labels': 12, 'batch_size': 16, 'lr': 2e-05, 'warmup': 0.2, 'train_size': 497, 'weight_decay': 0.001, 'n_epochs': 15, 'max_len': 120, 'dropout': 0.2}
